In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

# from research_tools.gpu import get_gpus_available
# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Loaded environment variables from .env file.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "LLM-LAT/zephyr7b-beta-rmu-lat-unlearn-wmdp-bio-cyber"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B"
model_id = "HuggingFaceH4/zephyr-7b-beta"

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/chat_template.jinja'


In [5]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq

dataset_config = DATASETS_DICT[Datasets.WMDP]

# retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")

retain_train = local_load_dataset(data_dir, dataset_config["retain_files"])
retain_val = local_load_dataset(data_dir, dataset_config["val_retain_files"])

unlearn_files = dataset_config["unlearn_files"]
val_unlearn_files = dataset_config["val_unlearn_files"]

n_val_files = 4
max_length = 512

forget_train_1 = local_load_dataset(data_dir, unlearn_files[:n_val_files])
forget_train_2 = local_load_dataset(data_dir, unlearn_files[n_val_files:])

forget_val_1 = local_load_dataset(data_dir, val_unlearn_files[:n_val_files])
forget_val_2 = local_load_dataset(data_dir, val_unlearn_files[n_val_files:])

forget_train_1_records = process_corpus(forget_train_1, tokenizer, max_length)
forget_train_2_records = process_corpus(forget_train_2, tokenizer, max_length)
retain_train_records = process_corpus(retain_train, tokenizer, max_length)
forget_train_records = forget_train_1_records + forget_train_2_records

forget_train_mcq_1_records = process_mcq(
    forget_val_1, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_2_records = process_mcq(
    forget_val_2, tokenizer, max_length, expand_choices=False
)
forget_train_mcq_records = forget_train_mcq_1_records + forget_train_mcq_2_records

forget_val_1_records = process_mcq(forget_val_1, tokenizer, max_length)
forget_val_2_records = process_mcq(forget_val_2, tokenizer, max_length)
retain_val_records = process_mcq(retain_val, tokenizer, max_length)
forget_val_records = forget_val_1_records + forget_val_2_records

Filter:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Filter:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
eval_dict = {
    "A": forget_val_1_records,
    "B": forget_val_2_records,
    "retain": retain_val_records,
}

In [5]:
import wandb

wandb.login()

config = {
    "model_id": model_id,
    "magnitude": 6.5,
    "lr": 1e-5,
    "n_epochs": 12,
    "forget_alphas": {"A": 0.39422},
    "retain_alphas": {"B": 13.51609, "retain": 1},
    "datasets_config": dataset_config,
}


run = wandb.init(
    project="relearn", config=config, tags=["rmu", "debug"], entity="12tqian"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 12tqian. Use `wandb login --relogin` to force relogin


In [6]:
# from relearn.unlearn.rmu import train_rmu

# # i retain using corpus because everything in unlearning should be corpus, no mcq that doesn't make sense
# # mcq should only be used to eval
# # i care if the model can speak english, not if it can answer mcq
# # force norm to be big so it's random things for weird stuff

# config = run.config

# model = train_rmu(
#     model,
#     {"A": forget_train_1_records},
#     {"B": forget_train_2_records, "retain": retain_train_records},
#     eval_records_dict=eval_dict,
#     n_epochs=config["n_epochs"],
#     magnitude=config["magnitude"],
#     lr=config["lr"],
#     forget_alphas=config["forget_alphas"],
#     retain_alphas=config["retain_alphas"],
#     eval_at_start=False,
#     max_batches=None,
#     verbose=True,
#     debug=False,
#     tokenizer=tokenizer,
# )

In [7]:
from pathlib import Path

path = Path("../models/wmdp") / "unlearn_A_retain_B"
# os.makedirs(path, exist_ok=True)
# model.save_pretrained(path)

# torch.save(model.state_dict(), path)
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16).to(
    device
)

# assert False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
from relearn.unlearn.rmu import train_rmu


model = train_rmu(
    model,
    {"B": forget_train_2_records},
    {"retain": retain_train_records},
    eval_records_dict=eval_dict,
    n_epochs=5,
    magnitude=6.5,
    lr=1e-5,
    forget_alphas={
        "B": 1,
    },
    retain_alphas={
        "retain": 1,
    },
    eval_at_start=False,
    verbose=True,
    max_batches=10,
    tokenizer=tokenizer,
)

  0%|          | 0/10 [00:00<?, ?it/s]/mnt/align1_drive/tcqian/unlearning_order/src/relearn/unlearn/rmu/unlearn.py:253: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 512, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  forget_loss = torch.nn.functional.mse_loss(
100%|██████████| 393/393 [00:42<00:00,  9.25it/s]


In [9]:
# text = "Who is the 44th president of the United States?"

# # Generate text
# inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
# output = model.generate(
#     **inputs,
#     do_sample=True,
#     max_length=100,
#     num_return_sequences=5,
#     temperature=0.7,
# )

# for i, o in enumerate(output):
#     print(f"Output {i+1}: {tokenizer.decode(o, skip_special_tokens=True)}")

In [12]:
path = Path("../models/wmdp") / "unlearn_A_unlearn_B"
os.makedirs(path, exist_ok=True)
model.save_pretrained(path)

[2025-01-04 15:39:59,489] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/mnt/align1_drive/tcqian/miniconda/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/mnt/align1_dri

In [13]:
run.finish()

A/acc,▁
B/acc,▁
B/forget_loss,█▇▆▅▂▁▁▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▁▁
global_step,▁▂▃▃▄▅▆▆▇█
retain/acc,▁
retain/retain_loss,▁▁▁▁▂▂▂▂█▂
A/acc,0.17357
B/acc,0.28662
B/forget_loss,0.03516
epoch,0


In [7]:
path = Path("../models/wmdp") / "unlearn_A_unlearn_B"
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16).to(
    device
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
from relearn.attacks.rtt import train_ft

# relearm only A
model = train_ft(
    model,
    10,
    forget_train_mcq_1_records,
    {
        "A": forget_val_1_records,
        "B": forget_val_2_records,
    },
    batch_size=2,
    lr=1e-6,
    eval_at_start=True,
    grad_accum_steps=2,
)

100%|██████████| 314/314 [00:21<00:00, 14.90it/s]


Start A Accuracy: 0.1735668789808917


100%|██████████| 79/79 [00:05<00:00, 14.68it/s]


Start B Accuracy: 0.28662420382165604


51it [00:09,  5.64it/s]

...

Epoch 9, Step 299, Loss 0.0002871573669835925


314it [00:58,  5.38it/s]
100%|██████████| 314/314 [00:20<00:00, 15.09it/s]


Epoch 9 A Accuracy: 0.9952229299363057


100%|██████████| 79/79 [00:05<00:00, 14.55it/s]


Epoch 9 B Accuracy: 0.39490445859872614
